In [ ]:
pip install -U scvi-tools

In [ ]:
pip install scanpy

In [ ]:
import os
import tempfile

import scanpy as sc
import scvi
import seaborn as sns
import torch
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
adata = sc.read("/work/SCVI_models/scAtlas_data/atlas_raw/scAtlas_Human_vascular_cells_processed_RAW_1.h5ad")

In [ ]:
adata

In [ ]:
adata.X.sum(0)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

split_key = "split"
adata.obs[split_key] = "train"
idx = list(range(len(adata)))
idx_train, idx_test = train_test_split(adata.obs_names, test_size=0.1, random_state=42)
adata.obs.loc[idx_train, split_key] = "train"
adata.obs.loc[idx_test, split_key] = "test"


# Filter the data to use only the training set and make a copy
adata_train = adata[adata.obs[split_key] == "train"].copy()
adata_test = adata[adata.obs[split_key] == "test"].copy()


In [ ]:
scvi.model.SCVI.setup_anndata(adata_train, layer=None, batch_key="donor_id")

In [ ]:
model = scvi.model.SCVI(adata_train, n_layers=2, n_latent=30, gene_likelihood="nb")

In [ ]:
model

In [ ]:
model.train()

In [ ]:
model.save("work/work/SCVI_models/may_models/SCVI_rerun_ATLAS_final_batch")

In [ ]:
model = scvi.model.SCVI.load("work/work/SCVI_models/may_models/SCVI_rerun_ATLAS_final_batch", adata=adata_train)

In [ ]:
library=adata_test.X.sum(axis = 1)
#library = adata_test.X.sum(axis = 1)

In [ ]:
y_pred = model.get_normalized_expression(adata_test, return_numpy=True)

In [ ]:
library = adata_test.X.sum(axis=1)

In [ ]:
library = np.array(library.flatten())

In [ ]:
y_true = adata_test.X.toarray() if hasattr(adata_test.X, 'toarray') else np.array(adata_test.X)

In [ ]:
rec = (library * y_pred.T ).T

In [ ]:
y_pred

In [ ]:
rec

In [ ]:
print(rec.shape)

In [ ]:
# If rec is an AnnData object, extract the X attribute (i.e., the data matrix)
import anndata
if isinstance(rec, anndata.AnnData):
    rec = rec.X

# Now, rec should be a numpy array or sparse matrix, which is what obsm expects
adata_test.obsm["X_reconstructed"] = rec

# Save the entire object with the reconstructed data
adata_test.write("adata_post_with_latent_and_reconstructed_SCALTAS_RERUN_finalbatch_2.h5ad")


In [ ]:
y_true = adata_test.X.toarray() if hasattr(adata_test.X, 'toarray') else np.array(adata_test.X)

In [ ]:
# Getting latent variables
latent = model.get_latent_representation(adata,batch_size = 256)
